In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import StandardScaler
from datetime import datetime

def create_weather_cluster(weather_code):
    """
    Clustert Wettercodes (00-99) in sinnvolle Kategorien basierend auf wahrnehmbaren Wetterphänomenen.
    
    Parameter:
    weather_code: int oder float - Wettercode zwischen 00 und 99
    
    Returns:
    str: Wettercluster-Name
    """
    if pd.isna(weather_code):
        return 'Fehlt'
    
    code = int(weather_code)
    
    if 0 <= code <= 3:
        return 'Bewölkung'
    elif 4 <= code <= 9:
        return 'Sichtminderung'
    elif 10 <= code <= 49:
        return 'Nebel'
    elif 50 <= code <= 79:
        return 'Niederschlag'
    elif 80 <= code <= 88:
        return 'Schauer'
    elif 89 <= code <= 94:
        return 'Gewitter'
    elif 95 <= code <= 98:
        return 'Extremwetter'
    elif code == 99:
        return 'Fehlt'
    else:
        return 'Unbekannt'

def get_temp_category(temp):
    """Kategorisiert Temperaturen in sinnvolle Bereiche."""
    if temp < 5: return 'sehr_kalt'
    elif temp < 10: return 'kalt'
    elif temp < 15: return 'mild'
    elif temp < 20: return 'warm'
    else: return 'heiss'

def get_wind_category(speed):
    """Kategorisiert Windgeschwindigkeiten (vereinfachte Beaufort-Skala)."""
    if speed < 5: return 'schwach'
    elif speed < 10: return 'maessig'
    elif speed < 15: return 'frisch'
    else: return 'stark'

def engineer_features(df):
    """
    Führt Feature Engineering für alle relevanten Daten durch.
    
    Parameter:
    df: pandas DataFrame mit Wetterdaten und Datum
    
    Returns:
    pandas DataFrame mit neuen Features
    """
    df_engineered = df.copy()
    
    # Datum zu datetime konvertieren falls nötig
    if not pd.api.types.is_datetime64_any_dtype(df_engineered['Datum']):
        df_engineered['Datum'] = pd.to_datetime(df_engineered['Datum'])
    
    # 1. Datum-basierte Features
    df_engineered['Quartal'] = df_engineered['Datum'].dt.quarter
    df_engineered['ist_wochenende'] = df_engineered['Datum'].dt.dayofweek.isin([5, 6]).astype(int)
    df_engineered['tag_des_jahres'] = df_engineered['Datum'].dt.dayofyear
    df_engineered['kalenderwoche'] = df_engineered['Datum'].dt.isocalendar().week
    
    # 2. Wetter-Features
    # Wetter-Cluster wie zuvor
    if 'Wettercode' in df.columns:
        df_engineered['wetter_cluster'] = df_engineered['Wettercode'].apply(create_weather_cluster)
        cluster_dummies = pd.get_dummies(df_engineered['wetter_cluster'], prefix='wetter')
        df_engineered = pd.concat([df_engineered, cluster_dummies], axis=1)
        
        df_engineered['ist_schlechtwetter'] = df_engineered['wetter_cluster'].isin([
            'Niederschlag', 'Schauer', 'Gewitter', 'Extremwetter'
        ]).astype(int)
        
        df_engineered['ist_sichtbehinderung'] = df_engineered['wetter_cluster'].isin([
            'Sichtminderung', 'Nebel'
        ]).astype(int)
    
    # Temperatur- und Wind-Kategorien
    if 'Temperatur' in df.columns:
        df_engineered['temperatur_kategorie'] = df_engineered['Temperatur'].apply(get_temp_category)
        temp_dummies = pd.get_dummies(df_engineered['temperatur_kategorie'], prefix='temp')
        df_engineered = pd.concat([df_engineered, temp_dummies], axis=1)
    
    if 'Windgeschwindigkeit' in df.columns:
        df_engineered['wind_kategorie'] = df_engineered['Windgeschwindigkeit'].apply(get_wind_category)
        wind_dummies = pd.get_dummies(df_engineered['wind_kategorie'], prefix='wind')
        df_engineered = pd.concat([df_engineered, wind_dummies], axis=1)
    
    # 3. Feature-Interaktionen
    # Wetter x Wochenende
    for weather_col in ['Bewoelkung', 'Temperatur', 'Windgeschwindigkeit']:
        if weather_col in df.columns:
            df_engineered[f'{weather_col}_wochenende'] = df_engineered[weather_col] * df_engineered['ist_wochenende']
    
    # Temperatur x Jahreszeit
    if 'Temperatur' in df.columns:
        seasons = [col for col in df_engineered.columns if col.startswith('Jahreszeit_')]
        for season in seasons:
            df_engineered[f'Temperatur_{season}'] = df_engineered['Temperatur'] * df_engineered[season]
    
    # 4. Feature Skalierung für numerische Features
    numeric_features = ['Bewoelkung', 'Temperatur', 'Windgeschwindigkeit', 'tag_des_jahres', 'kalenderwoche']
    numeric_features.extend([col for col in df_engineered.columns if '_wochenende' in col or col.startswith('Temperatur_Jahreszeit')])
    numeric_features = [col for col in numeric_features if col in df_engineered.columns]
    
    if numeric_features:
        scaler = StandardScaler()
        df_engineered[numeric_features] = scaler.fit_transform(df_engineered[numeric_features])
    
    return df_engineered

def main():
    """
    Hauptfunktion zum Laden, Verarbeiten und Speichern der Daten mit erweiterten Features.
    """
    # Pfade definieren
    input_path = '/workspaces/bakery_sales_prediction/2_BaselineModel/data/data_clean.csv'
    output_path = '/workspaces/bakery_sales_prediction/2_BaselineModel/data/data_feature_engineered.csv'
    
    try:
        # Daten laden
        print("Lade Daten...")
        df = pd.read_csv(input_path)
        print(f"Daten geladen: {df.shape[0]} Zeilen, {df.shape[1]} Spalten")
        
        # Zeige verfügbare Spalten
        print("\nVerfügbare Spalten:")
        print(df.columns.tolist())
        
        # Erweitertes Feature Engineering durchführen
        print("\nFühre erweitertes Feature Engineering durch...")
        df_engineered = engineer_features(df)
        
        # Zeige neue Features nach Kategorien
        new_columns = [col for col in df_engineered.columns if col not in df.columns]
        
        # Gruppiere neue Features nach Typ
        date_features = [col for col in new_columns if any(x in col for x in ['Quartal', 'wochenende', 'tag_des_jahres', 'kalenderwoche'])]
        weather_features = [col for col in new_columns if any(x in col for x in ['wetter_', 'temp_', 'wind_'])]
        interaction_features = [col for col in new_columns if '_wochenende' in col or col.startswith('Temperatur_Jahreszeit')]
        
        print(f"\nNeue Features erstellt: {len(new_columns)}")
        print("\nDatum-basierte Features:", date_features)
        print("\nWetter-Features:", weather_features)
        print("\nInteraktions-Features:", interaction_features)
        
        # Verteilungen der wichtigsten neuen Features
        print("\nVerteilungen:")
        if 'Quartal' in df_engineered.columns:
            print("\nQuartalsverteilung:")
            print(df_engineered['Quartal'].value_counts().sort_index())
        
        if 'ist_wochenende' in df_engineered.columns:
            print("\nWochenend-Verteilung:")
            print(df_engineered['ist_wochenende'].value_counts().map({0: 'Werktag', 1: 'Wochenende'}))
        
        if 'wetter_cluster' in df_engineered.columns:
            print("\nWetter-Cluster Verteilung:")
            print(df_engineered['wetter_cluster'].value_counts())
        
        # Speichere die Daten
        print(f"\nSpeichere Daten nach: {output_path}")
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        df_engineered.to_csv(output_path, index=False)
        print(f"Daten erfolgreich gespeichert: {df_engineered.shape[0]} Zeilen, {df_engineered.shape[1]} Spalten")
        
        # Zeige Beispiele der neuen Features
        print("\nBeispiele der wichtigsten neuen Features:")
        sample_features = ['Quartal', 'ist_wochenende', 'tag_des_jahres', 'kalenderwoche',
                         'temperatur_kategorie', 'wind_kategorie', 'wetter_cluster']
        sample_features = [f for f in sample_features if f in df_engineered.columns]
        if sample_features:
            print(df_engineered[sample_features].head())
        
    except FileNotFoundError:
        print(f"Fehler: Datei nicht gefunden unter {input_path}")
        print("Bitte überprüfen Sie den Pfad zur Eingabedatei.")
    except Exception as e:
        print(f"Fehler beim Verarbeiten der Daten: {str(e)}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()

Lade Daten...
Daten geladen: 9334 Zeilen, 48 Spalten

Verfügbare Spalten:
['id', 'Datum', 'Umsatz', 'KielerWoche', 'Bewoelkung', 'Temperatur', 'Windgeschwindigkeit', 'Wettercode', 'ist_feiertag', 'feiertag_vortag', 'feiertag_folgetag', 'Jahr', 'Wettercode_fehlt', 'Warengruppe_Brot', 'Warengruppe_Brötchen', 'Warengruppe_Croissant', 'Warengruppe_Konditorei', 'Warengruppe_Kuchen', 'Warengruppe_Saisonbrot', 'Wochentag_Monday', 'Wochentag_Saturday', 'Wochentag_Sunday', 'Wochentag_Thursday', 'Wochentag_Tuesday', 'Wochentag_Wednesday', 'Monat_2', 'Monat_3', 'Monat_4', 'Monat_5', 'Monat_6', 'Monat_7', 'Monat_8', 'Monat_9', 'Monat_10', 'Monat_11', 'Monat_12', 'Jahreszeit_Herbst', 'Jahreszeit_Sommer', 'Jahreszeit_Winter', 'Feiertag_Heiligabend', 'Feiertag_Kein_Feiertag', 'Feiertag_Ostermontag', 'Feiertag_Ostern', 'Feiertag_Pfingsten', 'Feiertag_Pfingstmontag', 'Feiertag_Silvester', 'Feiertag_Tag der Arbeit', 'Feiertag_Tag der deutschen Einheit']

Führe erweitertes Feature Engineering durch...

N